## 1.- Data loading:

In [ ]:
%matplotlib inline

import numpy as np
import xarray as xr
from matplotlib import pyplot as plt

ds = xr.open_dataset("Murrumbidgee_near_Bundure__MUR_B3.nc")
ds = ds.isel(x=slice(400,800), y=slice(0,400))

ds[['nbart_red','nbart_green','nbart_blue']].clip(0,2200).to_array().plot.imshow(col='time', col_wrap=6, robust=True)

In [ ]:
### This should be the start of the methodology

In [ ]:
#blue = ds.nbart_blue.astype(np.float32) / 1e4 
ds.where((ds.nbart_blue - ds.nbart_blue.quantile(0.25, dim='time'))<1000).rolling(time=7, min_periods=3, center=True).median()[['nbart_red','nbart_green','nbart_blue']].clip(0,2200).to_array().plot.imshow(col='time', col_wrap=6, robust=True)

In [ ]:
from skimage.morphology import dilation
from skimage.morphology import disk
from skimage.morphology import remove_small_objects
from sklearn.decomposition import PCA

def generate_bmask(ds):
    blue = ds.nbart_blue.astype(np.float32) / 1e4

    # 1.- Remove images with less than 1000 valid pixels
    blue = blue.isel(time=(np.count_nonzero(~np.isnan(blue.values), axis=(1,2)))>1000)

    # 2.- Create mask for reflectances with deviations more than 10% from lower quartile
    bmask = ((blue - blue.quantile(0.25, dim='time'))>.1).values

    # 3.- Remove small objects (< 25 pixels) and grow a 9 pixel disk buffer around remaining objects
    for i in range(bmask.shape[0]):
        bmask[i] = remove_small_objects(bmask[i], 25)
        bmask[i] = dilation(bmask[i], disk(9))
        bmask[i][np.isnan(blue[i].values)] = False
        
    # 4.- Wipe frames with more than 50% missing pixels
    #bmask[(np.count_nonzero(bmask, axis=(1,2)) / 160000)>.5] = np.nan

    return bmask


bmask = generate_bmask(ds)

ds[['nbart_red','nbart_green','nbart_blue']].where(~bmask).clip(0,2200).to_array().plot.imshow(col='time', col_wrap=6, robust=True)

In [ ]:
from skimage.morphology import dilation
from skimage.morphology import disk
from skimage.morphology import remove_small_objects
from sklearn.decomposition import PCA

def generate_bmask(ds):
    blue = ds.nbart_blue.astype(np.float32) / 1e4

    # 1.- Remove images with less than 1000 valid pixels
    blue = blue.isel(time=(np.count_nonzero(~np.isnan(blue.values), axis=(1,2)))>1000)
    blue = blue.where(blue<.45)

    # 2.- Create mask for reflectances with deviations more than 10% from lower quartile
    threshold = (blue.quantile(0.5, dim='time')*0.5).clip(min=0.05)
    bmask = ((blue - blue.quantile(0.5, dim='time'))>threshold).values

    # 3.- Remove small objects (< 25 pixels) and grow a 9 pixel disk buffer around remaining objects
    for i in range(bmask.shape[0]):
        bmask[i] = remove_small_objects(bmask[i], 25)
        bmask[i] = dilation(bmask[i], disk(9))
        bmask[i][np.isnan(blue[i].values)] = False
        
    # 4.- Wipe frames with more than 50% missing pixels
    #bmask[(np.count_nonzero(bmask, axis=(1,2)) / 160000)>.5] = np.nan

    return bmask


bmask = generate_bmask(ds)

ds[['nbart_red','nbart_green','nbart_blue']].where(~bmask).clip(0,2200).to_array().plot.imshow(col='time', col_wrap=6, robust=True)

In [ ]:
def generate_tmask(ds):
    bmask = generate_bmask(ds)
    blue = ds['nbart_blue'].astype(np.float32) / 1e4
    blue = blue.where(~bmask)
    diff_med_blue = blue - blue.rolling(time=7, min_periods=3, center=True).median()

    nir = ds['nbart_nir_1'].astype(np.float32) / 1e4
    nir = nir.where(~bmask)
    diff_med_nir = nir - nir.rolling(time=7, min_periods=3, center=True).median()
    
    swir = ds['nbart_swir_2'].astype(np.float32) / 1e4
    swir = swir.where(~bmask)
    diff_med_swir = swir - swir.rolling(time=7, min_periods=3, center=True).median()

    tbmask = np.abs(diff_med_blue.values)>0.03
    tnmask = (diff_med_nir.values<-0.1)
    tsmask = np.abs(diff_med_swir.values)>0.15
    
    tmask = tbmask + tnmask + tsmask
    
    print("-b-", np.count_nonzero(tbmask*tmask)/np.count_nonzero(tmask))
    print("-n-", np.count_nonzero(tnmask*tmask)/np.count_nonzero(tmask))
    print("-i-", np.count_nonzero(tsmask*tmask)/np.count_nonzero(tmask))
    
    print("-ub-", np.count_nonzero(tbmask*np.logical_not(tnmask+tsmask))/np.count_nonzero(tmask))
    print("-un-", np.count_nonzero(tnmask*np.logical_not(tbmask+tsmask))/np.count_nonzero(tmask))
    print("-ui-", np.count_nonzero(tsmask*np.logical_not(tbmask+tnmask))/np.count_nonzero(tmask))

    for i in range(tmask.shape[0]):
        tmask[i] = remove_small_objects(tmask[i], 9)
        tmask[i] = dilation(tmask[i], disk(9))

    return tmask

tmask = generate_tmask(ds)

ds[['nbart_red','nbart_green','nbart_blue']].where(~tmask*~bmask).clip(0,2200).to_array().plot.imshow(col='time', col_wrap=6, robust=True)

### t_blue mask contributes 56% unique values into the final mask (69% of the final mask)
### t_swir2 mask contributes 12% unique values into the final mask (37% of the final mask)
### t_nir1 mask contributes 7% unique values into the final mask (28% of the final mask)

In [ ]:
import xarray as xr
import numpy as np
from skimage.morphology import dilation
from skimage.morphology import disk
from skimage.morphology import remove_small_objects
from sklearn.decomposition import PCA


def generate_blue_mask(ds):
    blue = ds.nbart_blue.astype(np.float32) / 1e4

    # 1.- Remove images with less than 1000 valid pixels
    blue = blue.isel(time=(np.count_nonzero(~np.isnan(blue.values), axis=(1,2)))>1000)

    # 2.- Create mask for reflectances with deviations more than 10% from lower quartile
    qmask = ((blue - blue.quantile(0.25, dim='time'))>.1).values

    # 3.- Remove small objects (< 36 pixels) and grow a 15 pixel disk buffer around remaining objects
    for i in range(qmask.shape[0]):
        qmask[i] = remove_small_objects(qmask[i], 36)
        qmask[i] = dilation(qmask[i], disk(15))
        qmask[i][np.isnan(blue[i].values)] = False

    # 4.- Apply mask
    blue = blue.where(~qmask)

    # 5.- Discard frames with more than 33% missing pixels (relative to the initial valid pixels)
    #blue = blue.isel(time=(np.count_nonzero(qmask, axis=(1,2)) / (1+np.count_nonzero(~np.isnan(blue.values), axis=(1,2))))<.33)
    
    # 5.- Wipe frames with more than 33% missing pixels
    blue[(np.count_nonzero(qmask, axis=(1,2)) / (1+np.count_nonzero(~np.isnan(blue.values), axis=(1,2))))>.33] = np.nan
    
    # 6.- Compute temporal variability
    #temp_var = blue - blue.rolling(time=7, min_periods=3, center=True).median()
    
    # 7.- Define temporal variability threshold of 3%
    #tmask = np.abs(temp_var.values)>0.03
    
    # 8.- Grow a buffer around missing data
    #for i in range(tmask.shape[0]):
        #tmask[i] = dilation(tmask[i], disk(5))
    
    # 9.- Apply temporal mask
    #blue = blue.where(~tmask)
    
    # 10.- Wipe frames with more than 33% missing pixels
    #blue[np.count_nonzero(np.isnan(blue.values), axis=(1,2))>.33*400*400] = np.nan
    
    return blue


def generate_ts_mask(ds, var_name, thrs):

    da = ds[var_name].astype(np.float32) / 1e4
    
    blue = generate_blue_mask(ds)
    da = da.sel(time=blue.time).where(~np.isnan(blue))
    
    # 6.- Compute temporal variability
    ts_var = da - da.rolling(time=7, min_periods=3, center=True).median()
    
    # 7.- Define temporal variability threshold of 3%
    tmask = np.abs(ts_var.values)>thrs
    
    # 8.- Grow a buffer around missing data
    for i in range(tmask.shape[0]):
        tmask[i] = dilation(tmask[i], disk(5))
        
    return tmask

    
    # 9.- Apply temporal mask
    da = da.where(~tmask)
    
    # 10.- Wipe frames with more than 33% missing pixels
    da[np.count_nonzero(np.isnan(da.values), axis=(1,2))>.33*400*400] = np.nan
    
    return da


def stack_dataset(ds):
    stack = np.empty((0,400,400))
    for band_name in ds:
        band = ds[band_name].astype(np.float32) / 1e4

        # 1. Apply blue mask
        band = band.sel(time=blue.time).where(~np.isnan(blue))

        # 2.- Interpolate NaNs over time linearly
        band = band.interpolate_na(dim='time')

        # 3.- Interpolate NaNs at the start and end using nearest neighbor
        band = band.interpolate_na(dim='time', method='nearest', fill_value='extrapolate')

        # 4.- Apply median rolling filter along time (window=3)
        band = band.rolling(time=3, min_periods=1).median()

        stack = np.append(stack, band, axis=0)

    return stack.reshape(stack.shape[0], -1)

In [ ]:
%matplotlib inline

import numpy as np
import xarray as xr
from matplotlib import pyplot as plt

ds = xr.open_dataset("Murrumbidgee_near_Bundure__MUR_B3.nc")

ds = ds.isel(x=slice(400,800), y=slice(0,400))

blue = generate_blue_mask(ds)

#band = ds['nbart_red'].astype(np.float32) / 1e4
#band = band.sel(time=blue.time).where(~np.isnan(blue))

#band.plot(col='time', col_wrap=6)

In [ ]:
nir_ts_mask = generate_ts_mask(ds, 'nbart_nir_1', 0.05)
swir_ts_mask = generate_ts_mask(ds, 'nbart_swir_2', 0.05)
blue_ts_mask = generate_ts_mask(ds, 'nbart_blue', 0.03)

In [ ]:
i = 19
rgb = np.dstack((nir_ts_mask[i],swir_ts_mask[i],blue_ts_mask[i]))

plt.imshow(rgb.astype(np.float32))

In [ ]:
plt.imshow(nir_ts_mask[19], cmap='Reds')

In [ ]:
band = ds['nbart_swir_2'].astype(np.float32) / 1e4
band = band.sel(time=blue.time).where(~np.isnan(blue))

band.plot(col='time', col_wrap=6)

In [ ]:
ds[['nbart_red','nbart_green','nbart_blue']].clip(0,3500).to_array().plot.imshow(col='time', col_wrap=6, robust=True)

In [ ]:
from scipy.stats.mstats import rankdata

ranked = rankdata(blue.values, axis=0, use_missing=False)
ranked[np.isnan(blue.values)] = np.nan

ds['ranked'] = (['time','y','x'], ranked)

ds.ranked.plot(col='time', col_wrap=6, cmap='seismic', add_colorbar=False)

In [ ]:
blue = ds.nbart_blue.astype(np.float32) / 1e4
blue = blue.where(~bmask)

ds['diff_med_blue'] = blue - blue.rolling(time=7, min_periods=3, center=True).median()
ds.diff_med_blue.plot(col='time', col_wrap=6, cmap='seismic', add_colorbar=False)

In [ ]:
nir = ds['nbart_nir_1'].astype(np.float32) / 1e4
nir = nir.where(~bmask)

ds['diff_med_nir'] = nir - nir.rolling(time=7, min_periods=3, center=True).median()
ds.diff_med_nir.plot(col='time', col_wrap=6, cmap='seismic', add_colorbar=False)

In [ ]:
swir = ds['nbart_swir_2'].astype(np.float32) / 1e4
swir = swir.where(~bmask)

ds['diff_med_swir'] = swir - swir.rolling(time=7, min_periods=3, center=True).median()
ds.diff_med_swir.plot(col='time', col_wrap=6, cmap='seismic', add_colorbar=False)

In [ ]:
ds['diff_med_swir_th'] = (['time','y','x'], abs(ds['diff_med_swir'].values)>0.15)

ds.diff_med_swir_th.plot(col='time', col_wrap=6, add_colorbar=False)

In [ ]:
ds['diff_med_blue_th'] = (['time','y','x'], np.abs(ds['diff_med_blue'].values)>0.03)

ds.diff_med_blue_th.plot(col='time', col_wrap=6, add_colorbar=False)

In [ ]:
import numpy as np
import xarray as xr
from matplotlib import pyplot as plt

ds = xr.open_dataset("Murrumbidgee_near_Bundure__MUR_B3.nc")
ds = ds.isel(x=slice(400,800), y=slice(0,400))

blue = generate_blue_mask(ds)
diff_med_blue = blue - blue.rolling(time=7, min_periods=3, center=True).median()

nir = ds['nbart_nir_1'].astype(np.float32) / 1e4
nir = nir.where(~np.isnan(blue))

diff_med_nir = nir - nir.rolling(time=7, min_periods=3, center=True).median()




In [ ]:
tmask = (np.abs(diff_med_blue.values)>0.03) + (diff_med_nir.values<-0.1)

for i in range(tmask.shape[0]):
        tmask[i] = remove_small_objects(tmask[i], 9)
        tmask[i] = dilation(tmask[i], disk(9))

ds['tmask'] = (['time','y','x'], tmask)

ds.tmask.plot(col='time', col_wrap=6, add_colorbar=False)

In [ ]:
qmask = np.abs(ds['diff_med'].values)>0.03

for i in range(qmask.shape[0]):
        qmask[i] = dilation(qmask[i], disk(5))
        #qmask[i][np.isnan(blue[i].values)] = False
        
ds['diff_med_th_exp'] = (['time','y','x'], qmask)

ds.diff_med_th_exp.plot(col='time', col_wrap=6, add_colorbar=False)

In [ ]:
plt.figure(figsize=(20,10))

ax = plt.gca()
for _x in np.arange(0, 140, 6):
    ax.axvline(x=_x, ls='--',color='black')
    
plt.plot(np.vstack((mean,std)).T, 'o')

In [ ]:
plt.figure(figsize=(20,10))

ax = plt.gca()
for _x in np.arange(0, 140, 6):
    ax.axvline(x=_x, ls='--',color='black')
    
ax.axhline(y=0.005, ls='--',color='black')
    
plt.plot(std, 'o')